In [1]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report
)
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_excel('dataset-new.xlsx')
df = df[list(df)[:31]]

In [3]:
df

Corporate         Zone            Region                    Branch  \
0       NORTH        DELHI  DELHI NORTH WEST      DELHI-MANGOLPUR KALN   
1       NORTH        NORTH            KARNAL              ROORKEE-(UT)   
2       SOUTH    KARNATAKA            MYSORE       BELUR - HASSAN DIST   
3       NORTH      CENTRAL            NAGPUR                 REWA-(MP)   
4       SOUTH    TELANGANA         CYBERABAD     HYDERABAD - ERRAGADDA   
..        ...          ...               ...                       ...   
991     NORTH         WEST       NAVI MUMBAI            MUMBAI - THANE   
992     NORTH  BHUBANESWAR       BHUBANESWAR  CUTTACK - MAHANADI VIHAR   
993     NORTH        NORTH            KARNAL             PUNDRI - (HA)   
994     NORTH  BHUBANESWAR          VARANASI           SIDDHARTH NAGAR   
995     NORTH  BHUBANESWAR       BHUBANESWAR  CUTTACK - MAHANADI VIHAR   

     Branch Code   Lead ID Lead Product  Lead Amount Lead Rating  \
0           1311  42238330    Gold Loan         9500         Hot   
1           2695  41260593    Gold Loan        30000        Warm   
2           4046  41260595    Gold Loan        14000         Hot   
3           2919  42238329    Gold Loan        20000         Hot   
4           3703  42238322    Gold Loan         8400        Cold   
..           ...       ...          ...          ...         ...   
991         1010  41776763    Gold Loan         2000         Hot   
992         3535  41256697    Gold Loan         2000         Hot   
993         3288  41738700    Gold Loan         2000        Cold   
994         5038  41727678    Gold Loan        50000        Warm   
995         3535  41256701    Gold Loan         2000         Hot   

            Lead Source  ... Date of Birth       Age  Gender Purpose Category  \
0               Walk-in  ...       31048.0  36 Years  Female         Personal   
1               Walk-in  ...       35626.0  23 Years    Male         Personal   
2    Marketing Activity  ...       32143.0  33 Years    Male         Personal   
3        Sales Activity  ...       34201.0  27 Years    Male         Personal   
4    Marketing Activity  ...       33386.0  29 Years    Male         Personal   
..                  ...  ...           ...       ...     ...              ...   
991             Digital  ...           NaN       NaN    Male           Others   
992             Digital  ...           NaN       NaN     NaN         Personal   
993  Marketing Activity  ...           NaN       NaN     NaN         Business   
994         Call Centre  ...           NaN       NaN     NaN      Agriculture   
995             Digital  ...           NaN       NaN     NaN         Personal   

                         Occupation  Marital Status  Qualification  \
0                        House Wife         Married       Graduate   
1                     Self Employed         Married       Graduate   
2    Agriculture/Related Activities         Married         School   
3                          Salaried          Single       Graduate   
4                          Salaried         Married       Graduate   
..                              ...             ...            ...   
991                             NaN             NaN            NaN   
992                             NaN             NaN            NaN   
993                             NaN             NaN            NaN   
994                             NaN             NaN            NaN   
995                             NaN             NaN            NaN   

    Appointment Date  Assign To Name  Time In StatusCode  
0                NaN         PURNIMA                6 D   
1                NaN           SINGH               31 D   
2                NaN               N               31 D   
3                NaN            RUBY                6 D   
4                NaN     VINOD KUMAR                6 D   
..               ...             ...                 ...  
991              NaN         Shivani                2 D   
992         

In [4]:
list(df)

['Corporate',
 'Zone',
 'Region',
 'Branch',
 'Branch Code',
 'Lead ID',
 'Lead Product',
 'Lead Amount',
 'Lead Rating',
 'Lead Source',
 'Medium',
 'Lead Status Code(Text)',
 'Disbursed Amount',
 'Created By Code',
 'Lead Owner',
 'Lead Created On',
 'Lead Converted On',
 'Assign To Code',
 'Date of Purchase',
 'CIF No.',
 'Existing Customer',
 'Date of Birth',
 'Age',
 'Gender',
 'Purpose Category',
 'Occupation',
 'Marital Status',
 'Qualification',
 'Appointment Date',
 'Assign To Name',
 'Time In StatusCode']

In [5]:
# cat_column = ['Corporate',
#  'Zone',
#  'Lead Product',
#  'Lead Rating',
#  'Lead Source',
#  'Gender',
#  'Purpose Category',
#  'Occupation',
#  'Marital Status',
#  'Qualification','Region','Branch','Medium','Lead Owner']

In [6]:
df['Lead Status Code(Text)'] = df['Lead Status Code(Text)'].replace(['Disbursed', 'Rejected'],[1,0])

In [7]:
# df = pd.get_dummies(df, columns=col1)

In [8]:
# for col in ('Region','Branch','Medium','Lead Owner'):
#     df[col] = df[col].replace(df[col].unique(),list(range(len(df[col].unique()))))

# df['Lead Status Code(Text)'] = df['Lead Status Code(Text)'].replace(['Disbursed', 'Rejected'],[1,0])

In [9]:
df.isna().sum()

Corporate                   0
Zone                        0
Region                      0
Branch                      0
Branch Code                 0
Lead ID                     0
Lead Product                0
Lead Amount                 0
Lead Rating                 0
Lead Source                 0
Medium                      0
Lead Status Code(Text)      0
Disbursed Amount          497
Created By Code             0
Lead Owner                  0
Lead Created On             0
Lead Converted On         521
Assign To Code              0
Date of Purchase          520
CIF No.                     0
Existing Customer           0
Date of Birth             442
Age                       442
Gender                    339
Purpose Category            1
Occupation                447
Marital Status            498
Qualification             465
Appointment Date          669
Assign To Name              0
Time In StatusCode         16
dtype: int64

In [10]:
col_with_less_null = set(df.isna().sum()[df.isna().sum()<len(df)*0.1].index)

In [11]:
df = df[col_with_less_null]

In [12]:
cat_column = ['Branch',
 'Corporate',
 'Lead Product',
 'Lead Rating',
 'Lead Source',
 'Medium',
 'Purpose Category',
 'Region',
 'Zone']

In [13]:
column_to_del = ['Assign To Name', 'Assign To Code', 'Created By Code', 
                 'Existing Customer','CIF No.','Lead Created On','Lead ID','Branch Code',
                 'Lead Owner','Time In StatusCode']

In [14]:
imp_columns = set(list(df)) - set(column_to_del)

In [15]:
imp_columns

{'Branch',
 'Corporate',
 'Lead Amount',
 'Lead Product',
 'Lead Rating',
 'Lead Source',
 'Lead Status Code(Text)',
 'Medium',
 'Purpose Category',
 'Region',
 'Zone'}

In [16]:
df = df[imp_columns]

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
# df.select_dtypes(include=['object']).columns

In [19]:
X = df.drop('Lead Status Code(Text)',axis=1)
y = df['Lead Status Code(Text)']

In [20]:
all_cols = list(df)
all_cols.remove('Lead Status Code(Text)')
num_col = list(set(all_cols)-set(df.select_dtypes(include=['object']).columns))

In [21]:
num_col

['Lead Amount']

In [22]:
cat_column

['Branch',
 'Corporate',
 'Lead Product',
 'Lead Rating',
 'Lead Source',
 'Medium',
 'Purpose Category',
 'Region',
 'Zone']

In [23]:
num_pipeline = Pipeline([
        ('imputer', KNNImputer(n_neighbors=5)),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = num_col #list(housing_num)
cat_attribs = cat_column

# will be applying tranasformer object to the given subset of data
# NOTE: e.g., the "num_pipeline" transformer object will be applied on the given set of columns
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

 # sending the data through a chain of transformations
 # NOTE: the final dataset is a 2D numpy array containg all numerical data

In [24]:
X = full_pipeline.fit_transform(X)

In [25]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [26]:
X_train.shape , X_test.shape

((747, 804), (249, 804))

# Logistic regression

In [27]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [28]:
y_pred = classifier.predict(X_test)

In [29]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       120
           1       0.91      0.92      0.92       129

    accuracy                           0.91       249
   macro avg       0.91      0.91      0.91       249
weighted avg       0.91      0.91      0.91       249



In [30]:
pd.DataFrame({'Y_True':y_test,'Y_Pred':y_pred})

Y_True  Y_Pred
832       0       0
970       0       0
96        1       1
587       0       0
450       1       1
266       1       1
290       1       1
158       1       1
668       0       0
572       0       0
70        1       1
210       1       1
76        1       1
975       0       1
215       1       1
634       0       0
479       1       1
423       1       1
552       0       0
314       1       1
770       0       0
247       1       1
221       1       1
582       0       0
321       1       1
355       1       1
581       0       0
292       1       1
538       0       0
59        1       0
568       0       0
946       0       1
168       1       1
328       1       1
713       0       0
974       0       0
244       1       1
823       0       1
365       1       1
547       0       0
792       0       0
934       0       0
677       0       0
544       0       0
602       0       0
880       0       1
23        1       1
30        1       1
309       1       1
10        1       0
801       0       0
633       0       0
332       1       1
464       1       1
422       1       1
265       1       1
198       1       1
522       0       0
493       1       1
388       1       0
533       0       0
657       0       0
863       0       0
718       0       0
835       0       1
456       1       1
289       1       1
904       0       0
381       1       1
344       1       1
425       1       1
750       0       0
39        1       1
605       0       0
649       0       0
596       0       0
66        1       1
296       1       1
67        1       1
235       1       1
430       1       1
199       1       1
892       0       0
323       1       1
807       0       0
310       1       1
260       1       1
621       0       0
670       0       0
88        1       1
63        1       1
822       0       0
943       0       0
598       0       0
895       0       0
280       1       1
554       0       0
784       0       0
966       0       0
107       1       0
433       1       1
687       0       0
981       0       1
711       0       0
86        1       1
594       0       0
136       1       0
305       1       1
599       0       0
467       1       1
734       0       0
213       1       1
888       0       0
256       1       1
737       0       0
889       0       1
139       1       1
440       1       1
675       0       0
869       0       0
948       0       0
845       0       0
261       1       1
991       0       0
527       0       0
55        1       1
380       1       1
532       0       0
120       1       1
806       0       1
25        1       1
72        1       1
44        1       1
306       1       1
903       0       0
277       1       1
557       0       0
449       1       1
259       1       1
370       1       1
611       0       0
817       0       0
519       0       0
249       1       1
882       0       0
937       0       1
184       1       1
721       0       0
60        1       1
985       0       0
439       1       1
462       1       1
593       0       0
590       0       0
174       1       1
294       1       1
299       1       1
931       0       0
218       1       1
319       1       1
231       1       1
110       1       1
833       0       0
481       1       1
334       1       1
275       1       1
898       0       0
494       1       0
398       1       1
666       0       1
622       0       0
576       0       0
525       0       0
137       1       1
350       1       1
861       0       0
811       0       0
746       0       0
816       0       0
906       0       0
879       0       0
873       0       0
362       1       1
859       0       0
982       0       0
868       0       0
78        1       1
29        1       1
346       1       1
351       1       1
209       1       1
745       0       0
445       1       1
165       1       1
559       0       0
490       1       1
855       0       0
65        1       1
141       1       1
827  

In [31]:
print(round(accuracy_score(y_test,y_pred)*100,2))

91.16


# Decision Tree

In [32]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
print(accuracy_score(y_test,y_pred)*100)

91.96787148594377


In [35]:
accuracy_score(y_test,y_pred)*100

91.96787148594377

# Random forest

In [36]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [37]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)

92.3694779116466


# correlation

In [38]:
import numpy as np

In [39]:
df.corr().unstack().sort_values(ascending=False).drop_duplicates()

Lead Amount  Lead Amount               1.000000
             Lead Status Code(Text)    0.183083
dtype: float64

# columns required for prediction

In [40]:
# list(df)

imp_col = ['Branch',
 'Lead Product',
 'Purpose Category',
 'Region',
 'Medium',
 'Lead Source',
 'Lead Rating',
 'Zone',
 'Corporate',
 'Lead Amount']

In [41]:
X_for_prediction = df[imp_col].iloc[:2]
X_for_prediction

Branch Lead Product Purpose Category            Region       Medium Lead Source Lead Rating   Zone Corporate  Lead Amount
0  DELHI-MANGOLPUR KALN    Gold Loan         Personal  DELHI NORTH WEST  Bit Notices     Walk-in         Hot  DELHI     NORTH         9500
1          ROORKEE-(UT)    Gold Loan         Personal            KARNAL  Bit Notices     Walk-in        Warm  NORTH     NORTH        30000

In [42]:
X_processed = full_pipeline.transform(X_for_prediction)

In [43]:
X_processed

<2x804 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [44]:
# data = np.zeros((len(X_for_prediction),X_train.shape[1]))

In [45]:
# data[:,:X_processed.shape[1]] = X_processed

In [46]:
# data

In [47]:
y_pred = classifier.predict(X_processed)

In [48]:
X_for_prediction

Branch Lead Product Purpose Category            Region       Medium Lead Source Lead Rating   Zone Corporate  Lead Amount
0  DELHI-MANGOLPUR KALN    Gold Loan         Personal  DELHI NORTH WEST  Bit Notices     Walk-in         Hot  DELHI     NORTH         9500
1          ROORKEE-(UT)    Gold Loan         Personal            KARNAL  Bit Notices     Walk-in        Warm  NORTH     NORTH        30000

In [49]:
X_for_prediction['Predicted_Result'] = y_pred

In [50]:
X_for_prediction

Branch Lead Product Purpose Category            Region       Medium Lead Source Lead Rating   Zone Corporate  Lead Amount  Predicted_Result
0  DELHI-MANGOLPUR KALN    Gold Loan         Personal  DELHI NORTH WEST  Bit Notices     Walk-in         Hot  DELHI     NORTH         9500                 1
1          ROORKEE-(UT)    Gold Loan         Personal            KARNAL  Bit Notices     Walk-in        Warm  NORTH     NORTH        30000                 1